# Sunburst avec plotly.express

In [5]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

/opt/anaconda3/envs/dashenv/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [6]:
dfs = px.data.gapminder()
dfs = dfs.sample(100)
dfs.sample(5)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
563,Gambia,Africa,2007,59.448,1688359,752.749726,GMB,270
274,Chad,Africa,2002,50.525,8835739,1156.181860,TCD,148
1424,Spain,Europe,1992,77.570,39549438,18603.064520,ESP,724
1198,Paraguay,Americas,2002,70.755,5884491,3783.674243,PRY,600
823,Kenya,Africa,1987,59.339,21198082,1361.936856,KEN,404


In [7]:
fig = px.sunburst(dfs, path=["continent", "country"], 
                  values="pop", color="lifeExp",
                 color_continuous_scale='RdBu_r',)
fig.update_layout(margin=dict(t=0, b=10, r=10, l=10), 
                  template='plotly_dark',
                  paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [26]:
df_tips = px.data.tips()
df_tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [37]:
fig = px.sunburst(df_tips, path=['day', 'time', 'sex'],
                  values='total_bill', color='size',
                  color_continuous_scale='RdPu_r',)
fig.update_layout(margin=dict(t=0, b=0, r=0, l=0), 
                  template='plotly_dark',
                  paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [6]:
df_gapminder = px.data.gapminder().query("year == 2007")
df_gapminder.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32


In [15]:
fig = px.sunburst(df_gapminder, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_gapminder['lifeExp'], weights=df_gapminder['pop']))
fig.update_layout(margin=dict(t=10, b=10, r=10, l=10),
                  paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/sales_success.csv')
df.head()

,Unnamed: 0,region,county,salesperson,calls,sales
0,0,North,Dallam,JE,35,23
1,1,North,Dallam,ZQ,49,13
2,2,North,Dallam,IJ,20,6
3,3,North,Hartley,WE,39,37
4,4,North,Hartley,PL,42,37


In [9]:
levels = ['salesperson', 'county', 'region'] # levels used for the hierarchical chart
color_columns = ['sales', 'calls']
value_column = 'calls'

In [2]:
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees

In [3]:
df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)
average_score = df['sales'].sum() / df['calls'].sum()

NameError: name 'df' is not defined

In [4]:
fig.add_trace(go.Sunburst(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='RdBu',
        cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{color:.2f}',
    name=''
    ), 1, 1)
fig.update_layout(margin=dict(t=0, b=0, r=0, l=0), 
                  template="plotly_dark")

NameError: name 'fig' is not defined

In [66]:
df = pd.read_excel("/Users/new/Desktop/INFORMATIQUES/_Visualistion_de_données/data_world/emdat_public_2022_12_17_query_uid-SDA0A7.xlsx", skiprows=[0, 1, 2, 3, 4, 5])


/opt/anaconda3/envs/dashenv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [77]:
df.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1901-0003-BEL,1901,3,NaN,Technological,Technological,Industrial accident,Explosion,NaN,Coal mine,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
3,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
4,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


In [103]:
data = df.groupby(["Continent", "Region", "Disaster Group"])[["Total Deaths", "Total Affected"]].sum().reset_index()
v = data[["Total Deaths", "Total Affected"]] != 0

In [ ]:
import plotly.express as px
import numpy as np



fig = px.sunburst(data, path=["Continent", "Region", "Disaster Group"], values='Total Affected',
                  color='Total Deaths',
                  color_continuous_scale='Reds',
                  )
fig.update_layout(margin=dict(t=10, b=10, r=10, l=10),
                  paper_bgcolor='rgba(0,0,0,0)')
fig.show()